# Computational Theory Assessment

In [2]:
import numpy as np

PROBLEM 2: Include a method to calculate the cube root - Ian wants this

## <Strong>Problem 1 </strong>: Binary Words and Operations

### Parity 
**Parity** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Parity}(x, y, z) = x \oplus y \oplus z
$$

where ⊕ shows the **bitwise XOR** operation.

For each bit position, the result is 1 if an **odd number** of x, y and z contain a 1 in that position, otherwise 0.  
Parity acts as an **odd-bit detector**, ensuring that even a single bit change in the inputs alters the output.

**Simply Said:** Do I have an odd number of 1s(INPUT 1) or an even number of 1s(INPUT 0)?

**Why XOR is used**
- **XOR** stands for *exclusive OR*. 
- It's a logical operation that compares two bits and outputs **1 if they are different** and **0 if they are the same**.
- XOR is **fast**, **branch-free**, and supported directly in Hardware.  
- It implements addition mod 2 at the bit level.  

**Alternative forms in research**

Some implementations describe *Parity* differently:

| Formulation | Description | Why I Didn't Use|
| :-- | :-- | :-- |
| `(x + y + z) % 2` | Modulo-2 sum - Simplistic show of odd/even nature by summing bits and taking the remainder mod 2. |  You’d need to apply to each bit - expensive for 32-bit words. |
| `bin(x ^ y ^ z).count("1") % 2`  | Bit Count Method - parity of all bits within one number| Very slow, string conversion, not bitwise.|


**Truth Table (for 1-bit inputs)**

| x | y | z | Parity |
|:--:|:--:|:--:|:--:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

In [3]:
# Platform-defined integer type with 32 bits without sign
    # Uses np.uint32() : https://numpy.org/devdocs/user/basics.types.html
def parity(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the parity (XOR) of three 32-bit numbers.

    Each bit of the result is 1 if an odd number of the inputs
    have a 1 in that position, otherwise 0.

    Parameters:
        x (uint32): First 32-bit integer.
        y (uint32): Second 32-bit integer.
        z (uint32): Third 32-bit integer.

    Returns:
        np.uint32: The XOR (parity) of the three input values.
    """
    return np.uint32(x ^ y ^ z)


In [ ]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("x =", np.binary_repr(x, 8))
print("y =", np.binary_repr(y, 8))
print("z =", np.binary_repr(z, 8))

# Perform Parity
result = parity(x, y, z)

# Show result in binary
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Parity(x, y, z) =", np.binary_repr(result, 8))

x = 10101010
y = 11001100
z = 01111000
Parity(x, y, z) = 00011110


### Choose(x, y, z)

**Choose** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

Where:  
- `∧` is the **bitwise AND** operation,  
- `⊕` is the **bitwise XOR**,  
- `¬` is the **bitwise NOT** (flips every bit).  

**Objective**  
Choose uses `x` as a **bit-selector** - for every bit position:  
- if bit from x = 1, take the corresponding bit from y;  
- if bit from x = 0, take it from z.  

So x acts like a 32-bit “key” that chooses between y and z.


**Simply said:** **Choose** picks bits from `y` or `z` depending on whether `x`’s bit is 1 or 0.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (~x & z)` | OR instead of XOR | Non-Uniformity within problem set  |
| `z ^ (x & (y ^ z))` | Algebraic | Doesn't Match Spec, Not Great Readability |

**Why this method (`np.uint32((x & y) ^ (~x & z))`)?**  
- It exactly follows the standard equation.  
- **np.uint32** confines results to 32 bits (no overflow).  
- Bitwise operations are hardware-level fast and branch-free — each bit is processed independently.  


**Example (1-bit truth table)**  

| x | y | z | Choose(x,y,z) |
|:-:|:-:|:-:|:-------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [14]:
def choose(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the result of choosing bits from y and z based on x.
    
    For each bit position, if the bit in x is 1, take the bit from y,
    otherwise take the bit from z.

    bitwise NOT operator (~) flips each bit: 0 becomes 1, and 1 becomes 0.
    bitwise AND operator (&) compares each bit of two numbers and returns 1 if both bits are 1, otherwise returns 0.
    bitwise XOR operator (^) compares each bit of two numbers and returns 1 if the bits are different, otherwise returns 0.

    Parameters:
        x (uint32): The mask.
        y (uint32): The first number.
        z (uint32): The second number.
    """
    return np.uint32((x & y) ^ (~x & z))


In [15]:
# quick tests for Choose
print(choose(np.uint32(0), np.uint32(0), np.uint32(0)))   # expect 0
print(choose(np.uint32(0), np.uint32(0), np.uint32(1)))   # expect 1
print(choose(np.uint32(1), np.uint32(1), np.uint32(0)))   # expect 1
print(choose(np.uint32(1), np.uint32(1), np.uint32(1)))   # expect 1


0
1
1
1


### Maj(x, y, z)

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Maj}(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$


**Objective**  
The **Majority** function outputs 1 for each bit position where **two or more**
of the corresponding bits in `x`, `y`, and `z` are 1.
This makes it act as a “bitwise decider,” combining information from multiple inputs
to strengthen the *diffusion* of bits.


**Simply said:**  Majority - whichever value (0 or 1) occurs most often across x, y, and z.

**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (x & z) \| (y & z)` | Equivalent with OR instead of XOR | Less conventional - XOR keeps consistency with other SHA logical functions. |
| `(x & y) \| (z & (x \| y))` | Boolean simplification | Harder to read - hides the “majority” meaning behind extra grouping. |
| `((x ^ y) & z) \| (x & y)` | Optimized algebraic rearrangement |  Used for performance in C, unclear for teaching purposes & no scalability |


**Why this method (`np.uint32((x & y) ^ (x & z) ^ (y & z))`)?**  
- It directly mirrors the **official FIPS definition**, ensuring correctness and traceability.  
- Using `np.uint32` confines operations to 32-bit words, matching SHA-256’s internal word size.  
- The XOR version maintains consistency with the rest of SHA’s logic functions, which are all expressed with XOR, AND, and NOT.  
- It’s branch-free, efficient, and clear.

**Example (1-bit inputs)**  

| x | y | z | Maj(x,y,z) |
|:-:|:-:|:-:|:-----------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [ ]:
def Maj(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Compute the majority function.

    Maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)

    For each bit position, the output is 1 if two or more of
    the corresponding bits in x, y, and z are 1, otherwise 0.

    Parameters
    ----------
    x, y, z : np.uint32
        32-bit unsigned integers.

    Returns
    -------
    np.uint32
        The bitwise majority of x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)

In [8]:
# Simple Testing for Majority Function
x = np.uint32(0b1010)
y = np.uint32(0b1100)
z = np.uint32(0b0110)

print(np.binary_repr(Maj(x, y, z), 4)) # expect 1110


1110


In [ ]:
# Helper functions for Sigma functions
# Rotate-right operation for 32-bit words - Helper function for Sigma functions
def ROTR(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right operation for 32-bit words."""
    return np.uint32((x >> n) | (x << np.uint32(32 - n)))

# Shift-right operation for 32-bit words - Helper function for Sigma functions
def SHR(x: np.uint32, n: int) -> np.uint32:
    """Shift-right operation for 32-bit words."""
    return np.uint32(x >> n)


### Σ₀(x) — Sigma Zero

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_0^{\{256\}}(x) = \text{ROTR}^2(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)
$$

**Objective**  
The **Σ₀** function performs three right-rotations of the 32-bit `x`
by 2, 13, and 22 bits, then XORs the results.

This process ensures that bits from different positions in `x`
influence each other in later stages of the SHA-256 computation,
creating strong *diffusion* — a key cryptographic property
where small changes in input cause large, unpredictable changes in output.


**Simply Said:**  Σ₀(x) “scrambles” the bits of x by spinning them around and combining the three versions with XOR to mix them thoroughly.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `np.right_shift` with masking | Implements shifts manually | More verbose and less readable than `ROTR()` helper. |

**Why I Chose This Method**  
- The XOR-based rotation exactly matches the FIPS specification.  
- Using the shared `ROTR()` helper keeps all rotations consistent and clear across Σ₀, Σ₁, σ₀, and σ₁.  
- NumPy’s `np.uint32` enforces 32-bit wrapping automatically, avoiding overflow or sign issues.  
- It’s fast & readable.


**Behaviour Summary**
| Operation | Description |
|------------|-------------|
| ROTR²(x)  | Circular right rotation by 2 bits |
| ROTR¹³(x) | Circular right rotation by 13 bits |
| ROTR²²(x) | Circular right rotation by 22 bits |

The final result is the XOR (⊕) of these three rotated words.


In [10]:
def Sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₀ (Sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)

    Parameters
    ----------
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    -------
    np.uint32
        The resulting 32-bit word after applying Σ₀(x).
    """
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)


In [11]:
# Simple test for Sigma0
x = np.uint32(0b10010011)

# Display result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Original Number:", np.binary_repr(x, 32))
print("Sigma0 Result  :", np.binary_repr(Sigma0(x), 32))


Original Number: 00000000000000000000000010010011
Sigma0 Result  : 11000100100110100100110000100100


### Σ₁(x) — Sigma One

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_1^{\{256\}}(x) = \text{ROTR}^6(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)
$$


**Objective**  
The **Σ₁** function performs three right rotations of the 32-bit word `x`
by 6, 11, and 25 bits, then XORs the results.
This operation spreads information across bits and rounds, ensuring
that small differences in the input word produce large, unpredictable
differences in the output. It enhances *diffusion* and *non-linearity*,
both essential for cryptographic strength in SHA-256.

**Simply Said:** Σ₁(x) twists the bits of `x` three times by different amounts, then combines them.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `(x >> 6) ^ (x >> 11) ^ (x >> 25)` | Simple shifts instead of rotations | Loses information — bits are discarded instead of wrapped around. |
| `(ROTR(x,6) + ROTR(x,11) + ROTR(x,25)) & 0xFFFFFFFF` | Integer addition, not XOR | Incorrect — adds values numerically instead of mixing bits logically. |

---

**Why I Chose This Method**  
- The XOR-based rotation exactly matches the FIPS specification.  
- Using the shared `ROTR()` helper keeps all rotations consistent and clear across Σ₀, Σ₁, σ₀, and σ₁.  
- NumPy’s `np.uint32` enforces 32-bit wrapping automatically, avoiding overflow or sign issues.  
- It’s fast & readable.

**Behaviour Summary**
| Operation | Description |
|------------|--------------|
| ROTR⁶(x)  | Circular right rotation by 6 bits |
| ROTR¹¹(x) | Circular right rotation by 11 bits |
| ROTR²⁵(x) | Circular right rotation by 25 bits |

The final output is the XOR (⊕) of these three rotated words.


In [12]:
def Sigma1(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₁ (Sigma one) function.

    This function performs three right-rotations of a 32-bit word `x` by
    6, 11, and 25 bits, and returns the bitwise XOR of these results:

        Σ₁(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)

    Parameters
    ----------
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    -------
    np.uint32
        The resulting 32-bit word after applying Σ₁(x).
    """
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)


In [13]:
# Simple test for Sigma1
x = np.uint32(0b10010011)

# Display the result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print(np.binary_repr(Sigma1(x), 32))


01011110011000000100100110000010


### σ₀(x) — small sigma zero

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\sigma_0^{\{256\}}(x) = \text{ROTR}^7(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^3(x)
$$

---

**Objective**  
σ₀(x) is one of the *message schedule* functions used in SHA-256.  
It takes a 32-bit word x, rotates it right by 7 and 18 bits, then performs a simple right shift by 3 bits (filling with zeros).  

The three results are XORed together to create a new, highly mixed version of `x`.  
This helps expand the message into a sequence of words that feed the compression rounds, improving **diffusion** (how bit changes spread).

---

**Simply Said:**  σ₀(x) “scrambles” the bits of `x` by spinning them twice and sliding them once — the shift sprinkles in zeros to add more chaos.

---

**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `(ROTR(x,7) ^ ROTR(x,18) ^ (x >> 3))` | Canonical FIPS definition | ✅ Used — matches the SHA-256 specification exactly. |
| `(x >> 7) ^ (x >> 18) ^ (x >> 3)` | All logical shifts | Loses information — shifts discard bits instead of rotating them. |
| `(ROTR(x,7) + ROTR(x,18) + (x >> 3)) & 0xFFFFFFFF` | Integer addition, not XOR | Adds numerically; not bitwise — wrong operation. |
| `np.roll()` on bit arrays | Simulates rotation on arrays | Slower and unnecessary for fixed-width 32-bit words. |

---

**Why I Chose This Method**  
- Follows the exact equation defined in FIPS 180-4 for SHA-256.  
- Combines rotations and shifts correctly to mix and truncate bits as required for the message schedule.  
- Uses the shared `ROTR()` helper for consistent 32-bit rotation behaviour.  
- NumPy’s `np.uint32` keeps all arithmetic confined to 32 bits, matching the real hardware behaviour.  
- It’s simple, readable, and cryptographically faithful — no unnecessary complexity.


**Example (8-bit demo)**  
(Shown shorter for readability — real SHA-256 uses 32 bits.)

| Step | Operation | Result |
|:--|:--|:--|
| Input | x = `10010011` |  |
| 1 | ROTR⁷(x) | `11001001` |
| 2 | ROTR¹⁸(x) | `01100100` |
| 3 | SHR³(x) | `00010010` |
| XOR all | σ₀(x) | `10111111` |

The final output is the XOR (⊕) of these three results.


In [ ]:
def sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the σ₀ (sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        σ₀(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)

    This function performs two right-rotations followed by one right-shift,
    then XORS the results together.

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.
   
    Returns
    np.uint32
        The resulting 32-bit word after applying σ₀(x).
    """

    return np.bitwise_xor(np.bitwise_xor(ROTR(x, 7), ROTR(x, 18)), SHR(x, 3))

In [ ]:
#Testing

### Σ₀(x) — Sigma Zero

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_0^{\{256\}}(x) = \operatorname{ROTR}^{2}(x)\ \oplus\ \operatorname{ROTR}^{13}(x)\ \oplus\ \operatorname{ROTR}^{22}(x)
$$

**Objective**  
Performs three **bitwise right rotations** on a 32-bit word by 2, 13, and 22 bits, then XORs the results together.  
This helps *scramble* the input during the **main compression function** of SHA-256, spreading bit influence widely to achieve **diffusion**.


**Simply Said:**  
It’s a **bit blender** — spins the bits around the 32-bit word several times and mixes them together with XOR,  
so every input bit affects many output bits.


**Alternative forms and interpretations**  
- Sometimes written as  
  $$
  \Sigma_0(x) = (x \ggg 2)\ \oplus\ (x \ggg 13)\ \oplus\ (x \ggg 22)
  $$
  where $(\ggg)$ means *rotate right* (ROTR).  


**Why I chose this method**  
Using explicit rotations with XOR makes the implementation mirror the FIPS standard.  
Each rotation contributes to even diffusion across the 32-bit state.

I chose to use **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and efficient implementation.  
This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency,  
and makes the code easier to test and understand.


**Example (8-bit demo)**  

| Step | Operation | Result |
|:--|:--|:--|
| Input | x = `10010011` |  |
| 1 | ROTR²(x) | `11100100` |
| 2 | ROTR¹³(x) | `01111001` |
| 3 | ROTR²²(x) | `00100111` |
| XOR all | Σ₀(x) | `10111010` |

The final output is the XOR (⊕) of these three rotated results 


In [ ]:
def sigma1(x: np.uint32) -> np.uint32:
    """
    σ₁(x) — Lowercase sigma one 

    Computes: ROTRⁱ⁷(x) XOR ROTR¹⁹(x) XOR SHR¹⁰(x)

    Parameters
    x : np.uint32
        32-bit input word.

    Returns
    np.uint32
        32-bit result of σ₁(x).

    """
    return ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10)


In [ ]:
#Testing

## <strong>Problem 2 </strong>: Fractional Parts of Cube Roots

## <strong>Problem 3 </strong>: Padding

## <strong>Problem 4 </strong>: Hashes

## <strong>Problem 5 </strong>: Passwords